## Topic 2: QSVM4EO

This notebook will present the an example of using quantum-enhanced support vector machines (QSVM) for classification tasks on multi-spectral Earth Observation (EO) data. The main topics covered are introduction to classical SVM, where quantum computation could be used in the SVM calculations, the intricacies of preparing and encoding the classical data into useful quantum states and training QSVMs on gate-based quantum software simulators. Finally, some results on comparing classical and quantum-enhanced SVM models will be presented.

QSVM code is adapted from Qiskit QSVM example: https://qiskit-community.github.io/qiskit-machine-learning/tutorials/03_quantum_kernel.html

## Table of content

1. WIP 
2. WIP

## Introduction

### Use case

What we're doing and why

### Theory
Explain classical and quantum solutions...


<div style="background-color: #ffffff; padding: 10px; text-align: center; width: fit-content;">
  <img src="images/SVM_margin.png" width="300"><br>
  <span style="font-style: italic; font-size: 14px;">(Source: https://commons.wikimedia.org/wiki/File:SVM_margin.png)</span>
</div>



Dual Lagrangian formulation:

$$
\mathcal{L}(\boldsymbol{\alpha})=\sum_{n=1}^{N} \alpha_{n}-\frac{1}{2} \sum_{n=1}^{N} \sum_{m=1}^{N} y_{n} y_{m} \alpha_{n} \alpha_{m} \mathbf{x}_{n}^{\top} \mathbf{x}_{m}
$$


<div style="background-color: #ffffff; padding: 10px; text-align: center; width: fit-content;">
  <img src="images/SVM_kernel.png" width="600"><br>
  <span style="font-style: italic; font-size: 14px;">(Source: https://www.hackerearth.com/blog/developers/simple-tutorial-svm-parameter-tuning-python-r/
  )</span>
</div>

$$
\mathcal{L}(\boldsymbol{\alpha})=\sum_{n=1}^{N} \alpha_{n}-\frac{1}{2} \sum_{n=1}^{N} \sum_{m=1}^{N} y_{n} y_{m} \alpha_{n} \alpha_{m} \mathbf{z}_{n}^{\top} \mathbf{z}_{m}
$$

Some classical kernel functions:

$$
\begin{array}{l|l|l}
\hline \text { Name } & \text { Kernel } & \text { Hyperparameters } \\
\hline \text { Linear } & \mathbf{x}^{T} \mathbf{x}^{\prime} & - \\
\hline \text { Polynomial } & \left(\mathbf{x}^{T} \mathbf{x}^{\prime}+c\right)^{p} & p \in \mathbb{N}, c \in \mathbb{R} \\
\hline \text { Gaussian } & \mathrm{e}^{-\gamma\left\|\mathbf{x}-\mathbf{x}^{\prime}\right\|^{2}} & \gamma \in \mathbb{R}^{+} \\
\hline \text { Exponential } & \mathrm{e}^{-\gamma\left\|\mathbf{x}-\mathbf{x}^{\prime}\right\|} & \gamma \in \mathbb{R}^{+} \\
\hline \text { Sigmoid } & \tanh \left(\mathbf{x}^{T} \mathbf{x}^{\prime}+c\right) & c \in \mathbb{R} \\
\hline
\end{array}
$$

<div style="background-color: #ffffff; padding: 10px; text-align: center; width: fit-content;">
  <img src="images/SVM_QSVM_workflows.png" width="600"><br>
  <span style="font-style: italic; font-size: 14px;"></span>
</div>

## QSVM Tutorial

### Imports and setup

In [13]:
import qiskit

from qiskit_machine_learning.utils import algorithm_globals

algorithm_globals.random_seed = 12345

### Dataset

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

def read_csv(file_dir, selected_columns):
    df = pd.read_csv(file_dir)
    df_selected = df[selected_columns]
    X = df_selected.values
    Y = df['Label'].values
    return X,Y

In [2]:
train_data = 'data/train_32.csv'
test_data = 'data/test_32.csv'

# four_features_data = ['B02', 'B03', 'B04', 'B08']
# adhoc_dimension = 4
eight_features_data = ['B02', 'B03', 'B04', 'B08', 'NDVI', 'EVI', 'SAVI', 'NDWI']
adhoc_dimension = 8


In [3]:
train_features, train_labels = read_csv(train_data, eight_features_data)

test_features, test_labels = read_csv(test_data, eight_features_data)

# scaler = StandardScaler().fit(train_features)
# X_train_scaled = scaler.transform(train_features)
# X_test_scaled = scaler.transform(test_features)

In [4]:
from sklearn.svm import SVC

svc_classifier = SVC(kernel='linear')
svc_classifier.fit(train_features, train_labels)

train_score = svc_classifier.score(train_features, train_labels)
test_score = svc_classifier.score(test_features, test_labels)


print(f"Precomputed classical kernel classification test score: {test_score}")



Precomputed classical kernel classification test score: 0.5419921875


In [5]:
scaler = StandardScaler().fit(train_features)
train_features_scaled = scaler.transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [6]:
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel, FidelityStatevectorKernel

adhoc_feature_map = ZZFeatureMap(feature_dimension=adhoc_dimension, reps=1, entanglement="full")
#adhoc_feature_map = ZFeatureMap(feature_dimension=adhoc_dimension, reps=2)

#sampler = Sampler()

#fidelity = ComputeUncompute(sampler=sampler)

# adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)
adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)

adhoc_kernel.feature_map.decompose().draw()


┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     ├───┤├─────────────┤                                          └───┘»
q_3: ┤ H ├┤ P(2.0*x[3]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_4: ┤ H ├┤ P(2.0*x[4]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_5: ┤ H ├┤ P(2.0*x[5]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_6: ┤ H ├┤ P(2.0*x[6]) ├───────────────────────────────────────────────»
     ├───┤├─────────────┤                                               »
q_7: ┤ H ├┤ P(2.0*x[7]) ├───────────────────────────────────────────────»
     └───┘└─────────────┘                                               »
«                                                    »
«q_0: ──────────────────────────────────■─────────■──»
«                                       │         │  »
«q_1: ──────────────────────────────────┼────■────┼──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐  │  »
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├──┼──»
«     └──────────────────────────────┘└───┘└───┘┌─┴─┐»
«q_3: ──────────────────────────────────────────┤ X ├»
«                                               └───┘»
«q_4: ───────────────────────────────────────────────»
«                                                    »
«q_5: ───────────────────────────────────────────────»
«                                                    »
«q_6: ───────────────────────────────────────────────»
«                                                    »
«q_7: ───────────────────────────────────────────────»
«                                                    »
«                                                         »
«q_0: ───────────────────────────────────────■─────────■──»
«                                            │         │  »
«q_1: ──────────────────────────────────■────┼────■────┼──»
«     ┌──────────────────────────────┐┌─┴─┐  │    │    │  »
«q_2: ┤ P(2.0*(π - x[1])*(π - x[2])) ├┤ X ├──┼────┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐┌─┴─┐  │  »
«q_3: ┤ P(2.0*(π - x[0])*(π - x[3])) ├─────┤ X ├┤ X ├──┼──»
«     └──────────────────────────────┘     └───┘└───┘┌─┴─┐»
«q_4: ───────────────────────────────────────────────┤ X ├»
«                                                    └───┘»
«q_5: ────────────────────────────────────────────────────»
«                                                         »
«q_6: ────────────────────────────────────────────────────»
«                                                         »
«q_7: ────────────────────────────────────────────────────»
«                                                         »
«                                                              »
«q_0: ───────────────────────────────────────■──────────────■──»
«                                            │              │  »
«q_1: ──────────────────────────────────■────┼─────────■────┼──»
«                                       │    │         │    │  »
«q_2: ──────────────────────────────────┼────┼────■────┼────┼──»
«     ┌──────────────────────────────┐┌─┴─┐  │  ┌─┴─┐  │    │  »
«q_3: ┤ P(2.0*(π - x[1])*(π - x[3])) ├┤ X ├──┼──┤ X ├──┼────┼──»
«     ├──────────────────────────────┤└───┘┌─┴─┐└───┘┌─┴─┐  │  »
«q_4: ┤ P(2.0*(π - x[0])*(π - x[4])) ├─────┤ X ├─────┤ X ├──┼──»
«     └──────────────────────────────┘     └───┘     └───┘┌─┴─┐»
«q_5: ────────────────────────────────────────────────────┤ X ├»
«           

In [7]:
adhoc_matrix_train = adhoc_kernel.evaluate(x_vec=train_features)
adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=test_features, y_vec=train_features)

In [8]:
adhoc_svc = SVC(kernel="precomputed")

adhoc_svc.fit(adhoc_matrix_train, train_labels)

adhoc_score_precomputed_kernel = adhoc_svc.score(adhoc_matrix_test, test_labels)

print(f"Precomputed quantum kernel classification test score: {adhoc_score_precomputed_kernel}")

Precomputed quantum kernel classification test score: 0.330078125


In [9]:
matrix_train = adhoc_kernel.evaluate(x_vec=train_features_scaled)
adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=test_features_scaled, y_vec=train_features_scaled)

In [10]:
adhoc_svc = SVC(kernel="precomputed")

adhoc_svc.fit(adhoc_matrix_train, train_labels)

adhoc_score_precomputed_kernel = adhoc_svc.score(adhoc_matrix_test, test_labels)

print(f"Precomputed quantum kernel classification test score: {adhoc_score_precomputed_kernel}")

Precomputed quantum kernel classification test score: 0.4990234375


In [11]:
print(f'min: {train_features.min()}, max: {train_features.max()}')
print(f'min: {train_features_scaled.min()}, max: {train_features_scaled.max()}')

min: -29.769021739130437, max: 5064.0
min: -18.1594339837482, max: 16.512914583221786


In [12]:
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(train_features_scaled, train_labels)

train_score = svc_classifier.score(train_features_scaled, train_labels)
test_score = svc_classifier.score(test_features_scaled, test_labels)


print(f"Precomputed classical kernel classification test score: {test_score}")

Precomputed classical kernel classification test score: 0.5390625


## Exercise

Use the dataset with eight features and repeat the experiments above.

## Conclusions

Discuss real-world results and experience.

## References